1. 문서의 내용을 읽는다.
2. 문서를 쪼갠다
3. 임베딩 -> 벡터 데이터베이스에 저장
4. 질문이 있을때, 벡터 데이스베이스에서 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 함께 전달

In [5]:
%pip install -qU pypdf langchain-community


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
%pip install -qU langchain-text-splitters


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [10]:
%pip install chromadb
%pip install langchain
%pip install sentence-transformers
%pip install docx2txt


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
%pip install -qU langchain langchainhub 


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [11]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv

# 문서 불러오기
#file_path = "text.pdf"
#loader = PyPDFLoader(file_path)
#pages = loader.load()  # 기본 로드 방식 사용

# 문서 쪼개기
# chunk_size: 문서를 쪼개는 단위
# chunk_overlap: 문서를 쪼개는 단위의 겹치는 부분
# 겹치는 부분을 선언하는 이유는 원하는 문서를 가져올 확률을 높이기 위함
#text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=200)
#chunks = text_splitter.split_documents(pages)

# load_and_split 사용으로 문서 불러오기와 쪼개기를 한 번에 수행
# 분할기 정의
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=200)

# 문서 불러오기와 쪼개기를 한 번에 수행
# XXX 현재 pdf에서는 원하는 데이터를 가져오지 못함
# pdf에서의 원하는 데이터는 사진 형식의 표로 들어가 있기 때문.
# gpt에서 표를 이해하는 방식은 모델에 따라 다르지만 openAI는 markdown으로 이해하기에 markdown으로 변환이 필요하다.
file_path = "text.docx"
loader = Docx2txtLoader(file_path)
document_list = loader.load_and_split(text_splitter)


# 임베딩
load_dotenv()
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [12]:
from langchain_community.vectorstores import Chroma

# 25.03.25 현재 chroma db는 python 3.13 버전에서 작동하지 않음
# 벡터 데이터베이스 생성
# chroma db는 인 메모리 벡터 데이터베이스
database = Chroma.from_documents(
    documents=document_list,
    embedding=embeddings,
    persist_directory="chroma_db",
    collection_name="tax_collection"
)


In [13]:
# 유사도 검색
query = '연봉 3000만원인 직장인의 소득세는 얼마인가요?'
retrieved_docs = database.similarity_search(query, k=3)

In [14]:
retrieved_docs

[Document(metadata={'creationdate': '', 'creator': 'PyPDF', 'page': 8, 'page_label': '9', 'producer': 'Aspose.Words for .NET 25.1.0', 'source': 'text.pdf', 'total_pages': 170}, page_content='소득세법\n법제처 9\n 국가법령정보센터\n로서 대통령령으로 정하는 금액\n\u3000\u3000\u30001) 「발명진흥법」 제2조제2호에 따른 종업원등(이하 이 조, 제20조 및 제21조에서 “종업원등”이라 한다\n)이 같은 호에 따른 사용자등(이하 이 조에서 “사용자등”이라 한다)으로부터 받는 보상금. 다만, 보상금을 지\n급한 사용자등과 대통령령으로 정하는 특수관계에 있는 자가 받는 보상금은 제외한다.\n\u3000\u3000\u30002) 대학의 교직원 또는 대학과 고용관계가 있는 학생이 소속 대학에 설치된 「산업교육진흥 및 산학연\n협력촉진에 관한 법률」 제25조에 따른 산학협력단(이하 이 조에서 “산학협력단”이라 한다)으로부터 같은 법 \n제32조제1항제4호에 따라 받는 보상금\n저. 대통령령으로 정하는 복리후생적 성질의 급여\n처. 제20조제1항제6호에 따른 소득 중 다음의 요건을 모두 충족하는 소득으로서 대통령령으로 정하는 금액 \n이하의 금액\n1) 임원 또는 종업원(이하 이 조, 제20조 및 제164조의5에서 “임원등”이라 한다) 본인이 소비하는 것을 \n목적으로 제공받거나 지원을 받아 구입한 재화 또는 용역으로서 대통령령으로 정하는 기간 동안 재\n판매가 허용되지 아니할 것\n2) 해당 재화 또는 용역의 제공과 관련하여 모든 임원등에게 공통으로 적용되는 기준이 있을 것\n4. 연금소득 중 다음 각 목의 어느 하나에 해당하는 소득\n가. 「국민연금법」, 「공무원연금법」 또는 「공무원 재해보상법」, 「군인연금법」 또는 「군인 재해보상법\n」, 「사립학교교직원 연금법」, 「별정우체국법」 또는 「국민연금과 직역

In [15]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")

# 유사도 검색으로 가져온 문서를 LLM에 질문과 함께 전달
# 페르소나를 전달하면 답변이 더 잘나온다고 한다.
prompt = f"""[Identity]
- 당신은 한국의 소득세 전문가입니다.
- [Context]를 참고해서 사용자의 질문에 대답해주세요.

[Context]
{retrieved_docs}

[Question]
{query}
"""
response = llm.invoke(prompt)

In [16]:
response.content

'연봉 3000만원인 직장인의 소득세를 계산하기 위해서는 기본적으로 과세표준을 결정하고, 해당 과세표준에 해당하는 세율을 적용해야 합니다. 소득세 계산 과정은 다음과 같습니다:\n\n1. **총급여**: 연봉 3000만원\n\n2. **근로소득공제**: 근로소득공제는 총급여에 따라 차등 적용됩니다. 연봉 3000만원의 경우, 근로소득공제는 일반적으로 다음과 같은 방식으로 계산됩니다:\n    - 총급여가 3,300만원 이하인 경우: 총급여액 × 70%\n    예: 3000만원 × 0.7 = 2,100만원\n\n3. **과세표준**: 총급여에서 근로소득공제를 뺀 금액\n    - `과세표준 = 총급여 - 근로소득공제`\n    - 3,000만원 - 2,100만원 = 900만원\n\n4. **소득세 계산**: 2023년 기준 소득세 세율에 따르면,\n     - 과세표준 1,200만원 이하: 6%\n     - 따라서, 900만원 × 0.06 = 54만원\n\n5. **누진공제**: 누진 공제는 과세표준 구간에 따라 적용되지 않기 때문에 그대로 소득세 금액으로 봅니다.\n\n따라서, 연봉 3000만원인 직장인의 소득세는 약 54만원이 됩니다. 단, 실제 소득세액은 각종 세액 공제와 지방세 등을 추가로 고려해야 하므로, 실제 납부할 세금은 다를 수 있습니다. 세부적인 사항은 세무 전문가나 국세청 상담을 통해 확인하시는 것이 좋습니다.'

In [17]:
from langchain import hub

# question-answering prompt
# 기존에 이미 있는 프롬프트 활용
prompt = hub.pull("rlm/rag-prompt")

/Users/jys/code/study/study-llm-application/venv310/lib/python3.10/site-packages/langsmith/client.py:277: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [18]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [19]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)


In [17]:
ai_message = qa_chain.invoke({"query": query})

In [ ]:
ai_message